# Homework 4

**Due 10/11/2020.**


## References

+ Lecture 7.

## Instructions

+ Type your name and email in the "Student details" section below.
+ Develop the code and generate the figures you need to solve the problems using this notebook.
+ For the answers that require a mathematical proof or derivation you can either:
    
    - Type the answer using the built-in latex capabilities. In this case, simply export the notebook as a pdf and upload it on gradescope; or
    - You can print the notebook (after you are done with all the code), write your answers by hand, scan, turn your response to a single pdf, and upload on gradescope.

**Note**: 
+ Please match all the pages corresponding to each of the questions when you submit on gradescope. 

**Important:**
If you are running the notebook on Google Colab make sure you make a copy on your Google Drive so that you can resume your work later. To do this click on "File->Save a copy in Drive." Rename the copy as you wish by clicking on the filename you see on the very top. Then make sure you save regularly. If you close your browser, you can resume your work by going to your Google Drive and clicking on the notebook. You can find the notebooks the folder "My Drive/Colab Notebooks."

## Student details

+ **First Name:**
+ **Last Name:**
+ **Email:**

In [ ]:
# Here are some modules that you may need - please run this block of code:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_context('talk')
import numpy as np
import scipy
import scipy.stats as st

## Problem 1

This is a classic uncertainty propagation problem that you will have to solve using Monte Carlo sampling.
Consider the following stochastic harmonic oscillator:
$$
\begin{array}{ccc}
\ddot{y} + 2 \zeta \omega(X) \dot{y} + \omega^2(X)y &=& 0,\\
y(0) &=& y_0(X),\\
\dot{y}(0) &=& v_0(X),
\end{array}
$$
where:
+ $X = (X_1, X_2, X_3)$,
+ $X_i \sim N(0, 1)$,
+ $\omega(X) = 2\pi + X_1$, 
+ $\zeta = 0.01$,
+ $y_0(X) = 1+ 0.1 X_2$, and
+ $v_0 = 0.1 X_3$.

In words, this stochastic harmonic oscillator has an uncertain natural frequency and uncertain initial conditions.

Our goal is to propagate uncertainty through this dynamical system, i.e., estimate the mean and variance of its solution.
A solver for this dynamical system is given below:

In [ ]:
class Solver(object):
    def __init__(self, nt=100, T=5):
        """
        This is the initializer of the class.
        
        Arguments:
            nt - The number of timesteps.
            T  - The final time.
        """
        self.nt = nt
        self.T = T
        self.t = np.linspace(0, T, nt) # The timesteps on which we will get the solution
        # The following are not essential, but they are convenient
        self.num_input = 3             # The number of inputs the class accepts
        self.num_output = nt           # The number of outputs the class returns
        
    def __call__(self, x):
        """
        This special class method emulates a function call.
        
        Arguments:
            x - A 1D numpy array with 3 elements. This represents the stochastic input x = (x1, x2, x3).
        """
        ##uncertain quantities 
        x1 = x[0]
        x2 = x[1]
        x3 = x[2]
        
        #ODE parameters 
        omega = 2*np.pi + x1 
        y10 = 1 + 0.1*x2
        y20 = 0.1*x3
        y0 = np.array([y10, y20])   #initial conditions 
        
        #coefficient matrix 
        zeta = 0.01
        k = omega**2    ##spring constant
        c = 2*zeta*omega   ##damping coeff. 
        C = np.array([[0, 1],[-k, -c]])
        
        #RHS of the ODE system
        def rhs(y, t):
            return np.dot(C, y)
        
        y = scipy.integrate.odeint(rhs, y0, self.t)
        
        return y

Let's plot a few samples of the forward model to demonstrate how the solver works.

In [ ]:
# 1. Create the solver object
solver = Solver()
fig1, ax1 = plt.subplots()
ax1.set_xlabel('$t$ (Time)')
ax1.set_ylabel('$y(t)$ (Position)')
fig2, ax2 = plt.subplots()
ax2.set_xlabel('$t$ (Time)')
ax2.set_ylabel('$\dot{y}(t)$ (Velocity)')
for i in range(2):
    # Sample the random inputs (they are just standard normal)
    x = np.random.randn(solver.num_input) # solver.num_input is just 3
    # Evaluate the solver response
    y = solver(x) # This returns an (num timesteps) x (num states) array (100 x 2 here)
    # Plot the sample
    ax1.plot(solver.t, y[:, 0])
    ax2.plot(solver.t, y[:, 1], label='Sample {0:d}'.format(i+1))
plt.legend(loc=True)

For your convenience, here is code that takes many samples of the solver at once:

In [ ]:
def take_samples_from_solver(num_samples):
    """
    Takes ``num_samples`` from the ODE solver.
    
    Returns them in an array of the form: ``num_samples x 100 x 2`` (100 timesteps, 2 states (position, velocity))
    """
    samples = np.ndarray((num_samples, 100, 2))
    for i in range(num_samples):
        samples[i, :, :] = solver(np.random.randn(solver.num_input))
    return samples

It works like this:

In [ ]:
samples = take_samples_from_solver(50)
print(samples.shape)
print(samples)

### Part A 

Take 1000 samples of the solver output and plot the estimated mean position and velocity as a function of time.

In [ ]:
samples = take_samples_from_solver(1000)
# Sampled positions are: samples[:, :, 0]
# Sampled velocities are: samples[:, :, 1]
# Sampled position at the 10th timestep is: samples[:, 9, 0]
# etc.

# Your code here

### Part B

Monte Carlo produces a slightly different result every time you run it. This is epistemic uncertainty because of the finite number of samples used. Visualize this epistemic uncertainty about the mean response at $t=5$s as a function of the number of samples. To achieve this you need to rerun your Monte Carlo estimator for the mean response at $t=5$s many times, say 100 times and visualize each one of the samples you take.

**Solution**:

In [ ]:
# Your code here

### Part C
Repeat part A and B for the squared response.
That is, do exactly the same thing as above, but consider $y^2(t)$ and $\dot{y}^2(t)$ instead of $y(t)$ and $\dot{y}(t)$.
How many samples do you need to estimate the mean squared response at $t=5$s with negligible epistemic uncertainty?

**Solution**:

In [ ]:
# Your code here

### Part D

Now that you know how many samples you need to estimate the mean of the response and the square response, use the formula:
$$
\mathbb{V}[y(t)] = \mathbb{E}[y^2(t)] - \left(\mathbb{E}[y(t)]\right)^2,
$$
and similarly for $\dot{y}(t)$, to estimate the variance of the position and the velocity with negligible epistemic uncertainty.
Plot both quantities as a function of time.

**Solution**:

In [ ]:
# Your code here

### Part E

Put together the estimated mean and variance to plot a 95\% predictive interval for the position and the velocity as functions of time.

**Solution**:

In [ ]:
# Your code here